# Testing `pycic`

In [ ]:
import numpy as np
import pycic

import matplotlib.pyplot as plt
plt.style.use('ggplot')

: 

## Testing Power Spectrum Objects

Classes `cicPowerSpectrum`, which is used to hold the linear matter power (function of the scalar $k$) and `cicMeasPowerSpectrum` which is to hold the measured count-in-cells power spectrum.